In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import class_weight

In [ ]:
df = pd.read_csv("./instagram.csv")
df['class'] = df['class'].replace(['a', 'i', 's'], 'f')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df['class'] = df['class'].replace(['f'], 0)
df['class'] = df['class'].replace(['r'], 1)
print("Unique values in 'class' column:", df['class'].unique())

Unique values in 'class' column: [0 1]


<ipython-input-2-e333e0189604>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'] = df['class'].replace(['r'], 1)


In [ ]:
print(df.head())

    pos   flw   flg  bl  pic  lin    cl        cz     ni        erl   erc  \
0   120   126   740  69    1    0  1036  0.055556  0.111   8.600000  0.66   
1  1200   800  2800  91    1    1   381  0.000000  0.556   0.550000  0.03   
2     0     1    38   0    0    0     0  0.000000  0.000   0.000000  0.00   
3    49   297   275  44    1    1    36  0.222222  0.000  19.230000  0.58   
4     1  1800  7500   0    1    0   165  0.000000  0.000  29.780001  1.83   

      lt     hc     pr     fo        cs          pi  class  
0  0.000  0.556  0.667  0.278  0.173787  113.461594      0  
1  0.000  0.000  0.000  0.000  0.189783    7.124306      1  
2  0.000  0.000  0.000  0.000  1.000000    0.000000      0  
3  0.889  0.389  0.000  0.056  0.052664  111.983299      1  
4  0.000  0.000  0.000  0.000  0.000000    0.000000      0  


In [ ]:
X = df.drop('class', axis=1)
y = df['class']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.utils import class_weight

# Compute class weights explicitly for classes 0 and 1
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),
    y=y_train
)

# Explicitly define the dictionary with keys 0 and 1
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}
print("Class weights:", class_weights_dict)

Class weights: {0: 0.8278524503511289, 1: 1.262538147060517}


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
print("Unique classes in y_train:", np.unique(y_train))

Unique classes in y_train: [0 1]


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    # class_weight=class_weights_dict,
    # callbacks=[early_stopping]
)

Epoch 1/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7497 - loss: 0.5073 - val_accuracy: 0.8345 - val_loss: 0.3736
Epoch 2/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8243 - loss: 0.3902 - val_accuracy: 0.8511 - val_loss: 0.3250
Epoch 3/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8422 - loss: 0.3455 - val_accuracy: 0.8566 - val_loss: 0.3068
Epoch 4/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8481 - loss: 0.3270 - val_accuracy: 0.8605 - val_loss: 0.2992
Epoch 5/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8494 - loss: 0.3170 - val_accuracy: 0.8581 - val_loss: 0.2930
Epoch 6/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8536 - loss: 0.3105 - val_accuracy: 0.8629 - val_loss: 0.2894
Epoch 7/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8567 - loss: 0.3048 - val_accuracy: 0.8621 - val_loss: 0.2867
Epoch 8/50
2193/2193 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8546 - loss: 0.3063 

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8642 - loss: 0.2664
Test Accuracy: 0.8658


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.80      0.88     13257
           1       0.76      0.96      0.85      8670

    accuracy                           0.87     21927
   macro avg       0.87      0.88      0.86     21927
weighted avg       0.89      0.87      0.87     21927



In [ ]:
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.4f}")

Accuracy Score: 0.8658


In [ ]:
single_row_data = {
    'pos': 72,
    'flw': 210,
    'flg': 1400,
    'bl': 122,
    'pic': 1,
    'lin': 0,
    'cl': 35,
    'cz': 0.055555556,
    'ni': 0,
    'erl': 12.510000229,
    'erc': 0.2599999905,
    'lt': 0,
    'hc': 0.1669999957,
    'pr': 0,
    'fo': 1,
    'cs': 0.064790003,
    'pi': 16.430524826
}

# Convert the dictionary to a DataFrame
single_row_df = pd.DataFrame([single_row_data])

# Step 2: Scale the data using the same scaler as during training
single_row_scaled = scaler.transform(single_row_df)

# Step 3: Make the prediction
prediction = model.predict(single_row_scaled)

# Since the output is a probability (sigmoid), we threshold it to get the class (0 or 1)
predicted_class = (prediction > 0.5).astype("int32")

# Output the prediction
print(f"Predicted Class: {predicted_class[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
Predicted Class: 0
